In [ ]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp  # noqa: E402


import numpy as np  # noqa: E402
from asterias.core import (  # noqa: E402
    _download_data,
    _compute_all_profiles,
)

In [2]:
fs = _download_data(
    mh_lower_limit=0.0,
    mh_upper_limit=0.1,
    teff_lower_limit=5500,
    teff_upper_limit=5750,
    logg_lower_limit=4.0,
    logg_upper_limit=4.5,
    stellar_grid="kurucz",
    ld_data_path="/Users/cassese/Downloads/tmpdata",
    verbose=True,
)

Checking for/downloading 8 files


In [3]:
mus, coeffs = _compute_all_profiles(
    filepaths=fs,
    wavelength_ranges=jnp.array([[6000, 7000], [7000, 8000]]),
    filter_wavelengths=jnp.arange(5000, 10000),
    filter_throughput=jnp.ones(5000)
)
coeffs.shape

(8, 2, 17)

In [ ]:
import os
from asterias.core import _download
from asterias.constants import REMOTE_LD_DATA_PATH

supported_instruments = [
    "HST_STIS_G430L",
    "HST_STIS_G750L",
    "HST_WFC3_G280p1",
    "HST_WFC3_G280n1",
    "HST_WFC3_G102",
    "HST_WFC3_G141",
    "JWST_NIRSpec_Prism",
    "JWST_NIRSpec_G395H",
    "JWST_NIRSpec_G395M",
    "JWST_NIRSpec_G235H",
    "JWST_NIRSpec_G235M",
    # "JWST_NIRSpec_G140H", # not actually on the ExoTiC-LD server
    "JWST_NIRSpec_G140M-f100",
    "JWST_NIRSpec_G140H-f070",
    "JWST_NIRSpec_G140M-f070",
    "JWST_NIRISS_SOSSo1",
    "JWST_NIRISS_SOSSo2",
    "JWST_NIRCam_F322W2",
    "JWST_NIRCam_F444",
    "JWST_MIRI_LRS",
    "Spitzer_IRAC_Ch1",
    "Spitzer_IRAC_Ch2",
    "TESS",
]

ld_data_path = "/Users/cassese/Downloads/tmpdata"

def _read_sensitivity_data(mode):
    local_sensitivity_file_path = os.path.join(
        ld_data_path,
        "Sensitivity_files/{}_throughput.csv".format(mode))
    remote_sensitivity_file_path = os.path.join(
        REMOTE_LD_DATA_PATH,
        "Sensitivity_files/{}_throughput.csv".format(mode))

    # Check if exists locally.
    if not os.path.exists(local_sensitivity_file_path):
        _download(remote_sensitivity_file_path,
                    local_sensitivity_file_path)
        
    sensitivity_data = np.loadtxt(local_sensitivity_file_path,
                                    skiprows=1, delimiter=",")
    sensitivity_wavelengths = sensitivity_data[:, 0]
    sensitivity_throughputs = sensitivity_data[:, 1]

    return sensitivity_wavelengths, sensitivity_throughputs


for s in supported_instruments:
    _ = _read_sensitivity_data(s)